### Import Necessary Module and Set Path 

In [1]:
import sqlite3 as lite
import os
import sys
import collections
from pyne import nucname
sys.path.insert(0, '../scripts/')
jupyter_directory = os.getcwd()

In [2]:
import analysis as an

## Run Cyclus Input file

In [6]:
! rm cyclus.sqlite
! cyclus eg01-eg30-simple-newdep.xml

              :                                                               
          .CL:CC CC             _Q     _Q  _Q_Q    _Q    _Q              _Q   
        CC;CCCCCCCC:C;         /_\)   /_\)/_/\\)  /_\)  /_\)            /_\)  
        CCCCCCCCCCCCCl       __O|/O___O|/O_OO|/O__O|/O__O|/O____________O|/O__
     CCCCCCf     iCCCLCC     /////////////////////////////////////////////////
     iCCCt  ;;;;;.  CCCC                                                      
    CCCC  ;;;;;;;;;. CClL.                          c                         
   CCCC ,;;       ;;: CCCC  ;                   : CCCCi                       
    CCC ;;         ;;  CC   ;;:                CCC`   `C;                     
  lCCC ;;              CCCC  ;;;:             :CC .;;. C;   ;    :   ;  :;;   
  CCCC ;.              CCCC    ;;;,           CC ;    ; Ci  ;    :   ;  :  ;  
   iCC :;               CC       ;;;,        ;C ;       CC  ;    :   ; .      
  CCCi ;;               CCC        ;;;.      .C ;   

### The output file should be named 
         
          cyclus.sqlite

In [8]:
filename = 'poop.sqlite'
# Wait for the complete message to move on
cur = an.get_cursor(filename)
# get simulation time info
init_year, init_month, duration, timestep = an.get_timesteps(cur)

sfr_fuel = cur.execute('SELECT time, sum(quantity) FROM transactions '
                       'INNER JOIN resources ON resources.resourceid = '
                       'transactions.resourceid WHERE commodity = "sfr_fuel"'
                       ' GROUP BY time').fetchall()
print(max(an.get_timeseries(sfr_fuel, duration, True)))

mox = cur.execute('SELECT time, sum(quantity) FROM transactions '
                       'INNER JOIN resources ON resources.resourceid = '
                       'transactions.resourceid WHERE commodity = "mox"'
                       ' GROUP BY time').fetchall()
print(max(an.get_timeseries(mox, duration, True)))
fuel_stuff ={}
fuel_stuff['sfr_fuel'] = an.get_timeseries(sfr_fuel, duration, True)
fuel_stuff['mox'] = an.get_timeseries(mox, duration, True)
an.multi_line_plot(fuel_stuff, timestep, 'Years', 'Mass[MTHM]', 'Fuel', 'Fuel', init_year)

# get fuel mass timeseries
fuel_dict = an.fuel_usage_timeseries(cur, ['uox', 'mox', 'sfr_fuel'])
an.stacked_bar_chart(fuel_dict, timestep,
                  'Years', 'Mass[MTHM]',
                  'Total Fuel Mass vs Time',
                  'total_fuel',
                  init_year)


# prints net capacity and number of reactor timeseries
power_dict = an.get_power_dict(cur)
an.plot_power(cur)
ratio = {}
ratio['fr_to_lwr_ratio'] =[x/y for x, y in zip(power_dict['fr_inst'], power_dict['mox_lwr_inst'])]
an.stacked_bar_chart(ratio, timestep, 'Year', 'FR capacity / LWR capacity',
                     'Ratio vs Time', 'ratio', init_year)

1877.472
728.53
sink_source_facilities has no values
sink_source_facilities has no values


/home/teddy/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:36: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
# Number of Reactors Deployed
cur = an.get_cursor(filename)
lwr = cur.execute('SELECT count(*) FROM agententry '
                  'WHERE prototype = "lwr"').fetchone()
print('The Number of LWRs Deployed:')
print(lwr[0])

mox_lwr = cur.execute('SELECT count(*) FROM agententry '
                  'WHERE prototype = "mox_lwr"').fetchone()
print('The Number of MOX LWRs Deployed:')
print(mox_lwr[0])

sfr = cur.execute('SELECT count(*) FROM agententry '
                  'WHERE prototype = "fr"').fetchone()
print('The Number of FRs Deployed:')
print(sfr[0])

In [32]:
filename = 'cyclus.sqlite'
# Wait for the complete message to move on
cur = an.get_cursor(filename)
# get simulation time info
init_year, init_month, duration, timestep = an.get_timesteps(cur)

sfr_fuel = cur.execute('SELECT time, sum(quantity) FROM transactions '
                       'INNER JOIN resources ON resources.resourceid = '
                       'transactions.resourceid WHERE commodity = "sfr_fuel"'
                       ' GROUP BY time').fetchall()
print(max(an.get_timeseries(sfr_fuel, duration, True)))

mox = cur.execute('SELECT time, sum(quantity) FROM transactions '
                       'INNER JOIN resources ON resources.resourceid = '
                       'transactions.resourceid WHERE commodity = "mox"'
                       ' GROUP BY time').fetchall()
print(max(an.get_timeseries(mox, duration, True)))

# get fuel mass timeseries
fuel_dict = an.fuel_usage_timeseries(cur, ['uox', 'mox', 'sfr_fuel'])
an.stacked_bar_chart(fuel_dict, timestep,
                  'Years', 'Mass[MTHM]',
                  'Total Fuel Mass vs Time',
                  'total_fuel',
                  init_year)


# prints net capacity and number of reactor timeseries
power_dict = an.get_power_dict(cur)
an.plot_power(cur)
ratio = {}
ratio['fr_to_lwr_ratio'] =[x/y for x, y in zip(power_dict['fr_inst'], power_dict['mox_lwr_inst'])]
an.stacked_bar_chart(ratio, timestep, 'Year', 'FR capacity / LWR capacity',
                     'Ratio vs Time', 'ratio', init_year)


# get tailings mass timeseries
agent_ids = an.get_agent_ids(cur, 'enrichment')
tailings = an.facility_commodity_flux(cur, agent_ids,
                                      ['tailings'], True)
an.stacked_bar_chart(tailings, timestep,
                  'Year', 'Mass [MTHM]',
                  'Tailings vs Time',
                  'tailings', init_year)

# natural uranium demand
nat_u = collections.OrderedDict()
nat_u['nat_u'] = an.nat_u_timeseries(cur)
an.stacked_bar_chart(nat_u, timestep,
                     'Years', 'Natural Uranium Mass',
                     'Natural Uranium Demand vs Time',
                     'nat_u', init_year)

# uranium utilization
dictionary = {}
dictionary['uranium_utilization'] = an.u_util_calc(cur)
an.stacked_bar_chart(dictionary, timestep,
                     'Years', 'U Utilization Factor',
                     'U Utilization vs Time',
                     'u_util', init_year)

# get reprocessed material flow
agent_ids = an.get_agent_ids(cur, 'separations')
combined = an.facility_commodity_flux(cur, agent_ids, ['mox_TRU', 'uox_TRU', 'sfr_TRU'], True)
uox_TRU = an.facility_commodity_flux(cur, agent_ids, ['uox_TRU'], True)
mox_TRU = an.facility_commodity_flux(cur, agent_ids, ['mox_TRU'], True)
sfr_TRU = an.facility_commodity_flux(cur, agent_ids, ['sfr_TRU'], True)
an.stacked_bar_chart(combined, timestep, 'Years', 'Mass[MTHM]',
                  'Total reprocessed outflux vs Time', 'combined',
                  init_year)
an.stacked_bar_chart(uox_TRU, timestep, 'Years', 'Mass[MTHM]',
                  'reprocessed from spent UOX vs Time', 'from_spent_uox',
                  init_year)
an.stacked_bar_chart(mox_TRU, timestep, 'Years', 'Mass[MTHM]',
                  'reprocessed from spent MOX vs Time', 'from_spent_mox',
                  init_year)
an.stacked_bar_chart(sfr_TRU, timestep, 'Years', 'Mass[MTHM]',
                  'reprocessed from spent SFR Fuel vs Time', 'from_spent_sfr',
                  init_year)
print('Finished!')

6303.873
728.53
sink_source_facilities has no values
sink_source_facilities has no values


/home/dkadkf/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:32: RuntimeWarning: invalid value encountered in double_scalars
../scripts/analysis.py:857: RuntimeWarning: invalid value encountered in true_divide
  u_util_timeseries = np.nan_to_num(fuel_timeseries / u_supply_timeseries)


The Average Fuel Utilization Factor is: 
0.124854002726
Finished!


In [ ]:
# Number of Reactors Deployed
cur = an.get_cursor(filename)
lwr = cur.execute('SELECT count(*) FROM agententry '
                  'WHERE prototype = "lwr"').fetchone()
print('The Number of LWRs Deployed:')
print(lwr[0])

mox_lwr = cur.execute('SELECT count(*) FROM agententry '
                  'WHERE prototype = "mox_lwr"').fetchone()
print('The Number of MOX LWRs Deployed:')
print(mox_lwr[0])

sfr = cur.execute('SELECT count(*) FROM agententry '
                  'WHERE prototype = "fr"').fetchone()
print('The Number of FRs Deployed:')
print(sfr[0])

In [ ]:
# Fresh and Spent Fuel Composition
cur = an.get_cursor(filename)

sfr_fuel_qualid = cur.execute('SELECT qualid FROM transactions '
                              'INNER JOIN resources ON '
                              'resources.resourceid = transactions.resourceid '
                              'WHERE commodity = "sfr_fuel"').fetchone()
fresh_sfr_mox_comp = cur.execute('SELECT nucid, massfrac FROM compositions '
                                 'WHERE qualid = ' + str(sfr_fuel_qualid[0])).fetchall()
print('Composition for fresh sfr fuel: ')
for isotope in fresh_sfr_mox_comp:
    print(nucname.name(isotope[0]) + ': ' + str(isotope[1] * 100) + ' %')

print('\n \n')

sfr_waste_comp = cur.execute('SELECT nucid, massfrac FROM recipes '
                             'LEFT OUTER JOIN compositions '
                             'ON recipes.qualid = compositions.qualid '
                             'WHERE recipe = "sfr_waste_recipe"').fetchall()
print('Composition for sfr spent fuel: ')
for isotope in sfr_waste_comp:
    print(nucname.name(isotope[0]) + ': ' + str(isotope[1] * 100) + ' %')
    


mox_fuel_qualid = cur.execute('SELECT qualid FROM transactions '
                              'INNER JOIN resources ON '
                              'resources.resourceid = transactions.resourceid '
                              'WHERE commodity = "mox"').fetchone()
fresh_mox_comp = cur.execute('SELECT nucid, massfrac FROM compositions '
                             'WHERE qualid = ' + str(sfr_fuel_qualid[0])).fetchall()
print('Composition for fresh mox fuel: ')
for isotope in fresh_mox_comp:
    print(nucname.name(isotope[0]) + ': ' + str(isotope[1] * 100) + ' %')


mox_waste_comp = cur.execute('SELECT nucid, massfrac FROM recipes '
                             'LEFT OUTER JOIN compositions '
                             'ON recipes.qualid = compositions.qualid '
                             'WHERE recipe = "mox_waste_recipe"').fetchall()
print('Composition for mox spent fuel: ')
for isotope in mox_waste_comp:
    print(nucname.name(isotope[0]) + ': ' + str(isotope[1] * 100) + ' %')

In [ ]:
outstring = ''
outstring += 'Initial Year:' + str(init_year) + '\n'
outstring += 'Initial Month:' + str(init_month) + '\n'
outstring += 'Duration in Months:' + str(duration) + '\n'
outstring += 'End Year:' + str(int(init_year + duration/12)) + '\n'

outstring += '\n'

outstring += 'Total Fuel Used: [MTHM]' + '\n'
first = True
for key in fuel_dict:
    outstring += '\t' + key + ': ' + str(max(fuel_dict[key])) + '\n'
    if first:
        total = fuel_dict[key]
        first = False
    else:
        total = [x + y for x, y in zip(total, fuel_dict[key])]
outstring += '\t' + key + ': ' + str(max(total)) + '\n'

outstring += '\n'

outstring += 'Total Natural Uranium [MTHM]' + '\n'
outstring += '\t' + str(max(nat_u['nat_u'])) + '\n'

outstring += 'Total Tailings [MTHM]' + '\n'
outstring += '\t' + str(max(tailings['tailings'])) + '\n'

outstring += '\n'

outstring += 'Average Fuel Utilization Factor (Fuel/NatU):' + '\n'
outstring += '\t' + str(sum(dictionary['uranium_utilization']) / len(dictionary['uranium_utilization'])) + '\n'

outstring += '\n'

outstring += 'TRU reprocessed: [MTHM]' + '\n'
first = True
for key in combined:
    outstring += '\t' + key + ': ' + str(max(combined[key])) + '\n'
    if first:
        total = combined[key]
        first = False
    else:
        total = [x + y for x, y in zip(total, combined[key])]
outstring += '\t Total: ' + str(max(total)) + '\n'

outstring += '\n' 

outstring += 'Reactors Deployed:' + '\n'
outstring += '\t LWR: ' + str(lwr[0]) + '\n'
outstring += '\t FR: ' + str(sfr[0]) + '\n'
outstring += '\t MOX_LWR: ' + str(mox_lwr[0]) + '\n'

outstring += '\n'

outstring += 'Compositions' + '\n'
outstring += '\t Composition for fresh sfr fuel: ' + '\n'
for isotope in fresh_sfr_mox_comp:
    outstring += '\t\t ' + nucname.name(isotope[0]) + ': ' + str(isotope[1] * 100) + ' %' + '\n'
    
outstring += '\t Composition for fresh sfr spent fuel: ' + '\n'
for isotope in sfr_waste_comp:
    outstring += '\t\t' + nucname.name(isotope[0]) + ': ' + str(isotope[1] * 100) + ' %' + '\n'

outstring += 'Composition for fresh mox fuel: '
for isotope in fresh_mox_comp:
    outstring += '\t\t' + nucname.name(isotope[0]) + ': ' + str(isotope[1] * 100) + ' %' +'\n'
    
outstring += 'Composition for mox spent fuel: '
for isotope in mox_waste_comp:
    outstring +=  '\t\t' + nucname.name(isotope[0]) + ': ' + str(isotope[1] * 100) + ' %' + '\n'
    
print(outstring)
if 'simulation_results':
    with open('simulation_results', 'w') as output:
        output.write(outstring)

### Plots

In [ ]:
# Display Net Capacity vs Time
from IPython.display import Image
Image(filename='power_plot.png')

# Display Number of Reactors vs Time
from IPython.display import Image
Image(filename='number_plot.png')

# Display Tailings vs Time
from IPython.display import Image
Image(filename='tailings.png')

# Display Total Fuel vs Time
from IPython.display import Image
Image(filename='total_fuel.png')

# Display Uranium Utilization vs Time
from IPython.display import Image
Image(filename='u_util.png')

# Display Separated Pu Flow
from IPython.display import Image
Image(filename='combined.png')

from IPython.display import Image
Image(filename='mox_Pu.png')

from IPython.display import Image
Image(filename='uox_Pu.png')